In [290]:
import pandas as pd
from implicit.nearest_neighbours import CosineRecommender
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [291]:
df = pd.read_csv("rating2.csv")
df.head()

,users,movies,ratings
0,Lisa Rose,Lady in the Water,2.5
1,Lisa Rose,Snakes on a Plane,3.5
2,Lisa Rose,Just My Luck,3.0
3,Lisa Rose,Superman Returns,3.5
4,Lisa Rose,"You, Me and Dupree",2.5


In [292]:
utility = df.pivot(columns="movies", index="users", values="ratings")
utility

movies,Just My Luck,Lady in the Water,Snakes on a Plane,Superman Returns,The Night Listener,"You, Me and Dupree"
users,,,,,,
Claudia Puig,3.0,NaN,3.5,4.0,4.5,2.5
Gene Seymour,1.5,3.0,3.5,5.0,3.0,3.5
Jack Matthews,NaN,3.0,4.0,5.0,3.0,3.5
Lisa Rose,3.0,2.5,3.5,3.5,3.0,2.5
Michael Phillips,NaN,2.5,3.0,3.5,4.0,NaN
Mick LaSalle,2.0,3.0,4.0,3.0,3.0,2.0
Toby,NaN,NaN,4.5,4.0,NaN,1.0


In [293]:
pd.DataFrame(cosine_similarity(utility.fillna(0)))

,0,1,2,3,4,5,6
0,1.000000,0.884789,0.834622,0.928841,0.807057,0.885658,0.702841
1,0.884789,1.000000,0.982325,0.960646,0.862337,0.955691,0.770024
2,0.834622,0.982325,1.000000,0.906551,0.880155,0.928989,0.805550
3,0.928841,0.960646,0.906551,1.000000,0.838220,0.981831,0.712501
4,0.807057,0.862337,0.880155,0.838220,1.000000,0.891702,0.683164
5,0.885658,0.955691,0.928989,0.981831,0.891702,1.000000,0.734179
6,0.702841,0.770024,0.805550,0.712501,0.683164,0.734179,1.000000


In [294]:
df = df.fillna(0)
df["users"] = df["users"].astype("category")
df["movies"] = df["movies"].astype("category")
df

,users,movies,ratings
0,Lisa Rose,Lady in the Water,2.5
1,Lisa Rose,Snakes on a Plane,3.5
2,Lisa Rose,Just My Luck,3.0
3,Lisa Rose,Superman Returns,3.5
4,Lisa Rose,"You, Me and Dupree",2.5
5,Lisa Rose,The Night Listener,3.0
6,Gene Seymour,Lady in the Water,3.0
7,Gene Seymour,Snakes on a Plane,3.5
8,Gene Seymour,Just My Luck,1.5
9,Gene Seymour,Superman Returns,5.0


In [309]:
ratings = coo_matrix(
    (df["ratings"].astype(float), (df["users"].cat.codes, df["movies"].cat.codes))
)
ratings

<COOrdinate sparse matrix of dtype 'float64'
	with 35 stored elements and shape (7, 6)>

In [310]:
from implicit.cpu.als import AlternatingLeastSquares

model = AlternatingLeastSquares()
model.fit(ratings.tocsr())

  0%|          | 0/15 [00:00<?, ?it/s]

In [311]:
toby = 6
ids, scores = model.recommend(toby, ratings.tocsr()[toby], N=5)
ids, scores

(array([4, 0, 1, 3, 2], dtype=int32),
 array([ 2.9753819e-03, -4.0078610e-03, -4.4225454e-03, -3.4028235e+38,
        -3.4028235e+38], dtype=float32))

In [312]:
df.users.cat.categories[ids]

Index(['Michael Phillips', 'Claudia Puig', 'Gene Seymour', 'Lisa Rose',
       'Jack Matthews'],
      dtype='object')

In [319]:
model = CosineRecommender()
model.fit(ratings.tocsr())

/Users/alextanhongpin/Documents/go/recommender-learn/.venv/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 6.389617919921875e-05 seconds
  warnings.warn(


  0%|          | 0/6 [00:00<?, ?it/s]

In [320]:
toby = 6
ids, scores = model.recommend(toby, ratings.tocsr()[toby], N=5)
ids, scores

(array([4, 1, 0, 5, 3], dtype=int32),
 array([8.28976852, 7.83288013, 6.64235501, 0.        , 0.        ]))

In [321]:
df.users.cat.categories[ids]

Index(['Michael Phillips', 'Gene Seymour', 'Claudia Puig', 'Mick LaSalle',
       'Lisa Rose'],
      dtype='object')